In [4]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

from datetime import datetime

import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.layers as Layer

import tensorboard
import matplotlib.pyplot as plt

import numpy as np

import keras_tuner as kt

import time

physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)



In [2]:
# load the dataset and normalize
(train_images, train_labels), (test_images, test_labels) = keras.datasets.mnist.load_data()
train_images = train_images / 255.0

x_train = train_images.reshape(-1, 28, 28, 1) #add an additional dimension to represent the single-channel
x_test = test_images.reshape(-1, 28, 28, 1)

np.random.seed(1)
tf.random.set_seed(1)

# Regular Model -- Keras-Tuner

In [ ]:
LOG_DIR = f"{int(time.time())}"

def build_model(hp):
    regular_model = tf.keras.models.Sequential(name='CNN_Increasing_Filters')

    regular_model.add(Layer.Conv2D(8, (3, 3), padding='same', activation='relu')) 
    regular_model.add(Layer.Conv2D(9, (3, 3), padding='same', activation='relu'))
    regular_model.add(Layer.Conv2D(10, (3, 3),padding='same', activation='relu'))
    regular_model.add(Layer.MaxPooling2D(pool_size=(2, 2), strides=(2,2), name='Pooling_Layer_14x14'))

    regular_model.add(Layer.Conv2D(20, (3, 3), padding='same', activation='relu'))
    regular_model.add(Layer.Conv2D(21, (3, 3), padding='same', activation='relu'))
    regular_model.add(Layer.Conv2D(22, (3, 3),padding='same', activation='relu'))
    regular_model.add(Layer.MaxPooling2D(pool_size=(2, 2), strides=(2,2), name='Pooling_Layer_7x7'))

    regular_model.add(Layer.Conv2D(44, (3, 3), padding='same', activation='relu'))
    regular_model.add(Layer.Conv2D(45, (3, 3), padding='same', activation='relu'))
    regular_model.add(Layer.Conv2D(46, (3, 3), padding='same', activation='relu'))
    regular_model.add(Layer.MaxPooling2D(pool_size=(2, 2), strides=(2,2), name='Pooling_Layer_3x3'))

    # tuned
    regular_model.add(Layer.Conv2D(hp.Int('conv10_filters', min_value=160, max_value=160, step=20), (3, 3), padding='same', activation='relu'))

    regular_model.add(Layer.Flatten())
    # tuned
    regular_model.add(Layer.Dense(hp.Int('hidden_neurons', min_value=80, max_value=80)))
    regular_model.add(Layer.Activation('relu'))
    regular_model.add(Layer.Dense(10))
    regular_model.add(Layer.Activation('softmax'))
    
    # tuned
    optimizer=hp.Choice('optimizer', values=['adam', 'RMSprop'])
    # tuned
    keras.optimizers.Adam(hp.Choice('learning_rate', [.01, .001, .0001]))

    regular_model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return regular_model

# Define the Keras TensorBoard callback.
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=30,  # how many model variations to test?
    executions_per_trial=1,  # how many trials per variation? (same model could perform differently)
    directory=LOG_DIR)

tuner.search_space_summary()
    
tuner.search(x=x_train,
            y=train_labels,
            verbose=2,
            epochs=1,
            batch_size=512,
            callbacks=[tensorboard_callback],
            validation_data=(x_test, test_labels))

tuner.results_summary()

# Inverted Model -- Keras-Tuner

In [ ]:
LOG_DIR = f"{int(time.time())}"

def build_model(hp):

    inverted_model = tf.keras.models.Sequential(name='CNN_Decreasing_Filters')

    inverted_model.add(Layer.Conv2D(80, (3, 3), padding='same', activation='relu')) 
    inverted_model.add(Layer.Conv2D(75, (3, 3), padding='same', activation='relu'))
    inverted_model.add(Layer.Conv2D(70, (3, 3),padding='same', activation='relu'))
    inverted_model.add(Layer.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

    inverted_model.add(Layer.Conv2D(55, (3, 3), padding='same', activation='relu'))
    inverted_model.add(Layer.Conv2D(50, (3, 3), padding='same', activation='relu'))
    inverted_model.add(Layer.Conv2D(45, (3, 3),padding='same', activation='relu'))
    inverted_model.add(Layer.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

    inverted_model.add(Layer.Conv2D(25, (3, 3), padding='same', activation='relu'))
    inverted_model.add(Layer.Conv2D(20, (3, 3), padding='same', activation='relu'))
    inverted_model.add(Layer.Conv2D(15, (3, 3), padding='same', activation='relu'))
    inverted_model.add(Layer.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

    inverted_model.add(Layer.Conv2D(hp.Int('conv10_filters', min_value=10, max_value=10), (3, 3), padding='same', activation='relu'))

    inverted_model.add(Layer.Flatten())
    inverted_model.add(Layer.Dense(hp.Int('hidden_neurons', min_value=100, max_value=100)))
    inverted_model.add(Layer.Activation('relu'))
    inverted_model.add(Layer.Dense(10))
    inverted_model.add(Layer.Activation('softmax'))
    
    # tuned
    optimizer = hp.Choice('optimizer', values=['adam', 'RMSprop'])
    # tuned
    keras.optimizers.Adam(hp.Choice('learning_rate', [.01, .001, .0001]))

    inverted_model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return inverted_model


# Define the Keras TensorBoard callback.
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=100,  # how many model variations to test?
    executions_per_trial=1,  # how many trials per variation? (same model could perform differently)
    directory=LOG_DIR)

tuner.search_space_summary()
    
tuner.search(x=x_train,
            y=train_labels,    
            verbose=2,
            epochs=1,
            batch_size=1024,
            callbacks=[tensorboard_callback],
            validation_data=(x_test, test_labels))

tuner.results_summary()


# Hourglass Model -- Keras-Tuner

In [ ]:
LOG_DIR = f"{int(time.time())}"

def build_model(hp):

    hourglass_model = tf.keras.models.Sequential(name='CNN_HourGlass_Filters')

    hourglass_model.add(Layer.Conv2D(30, (3, 3), padding='same', activation='relu')) 
    hourglass_model.add(Layer.Conv2D(31, (3, 3), padding='same', activation='relu'))
    hourglass_model.add(Layer.Conv2D(32, (3, 3),padding='same', activation='relu'))
    hourglass_model.add(Layer.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

    hourglass_model.add(Layer.Conv2D(60, (3, 3), padding='same', activation='relu'))
    hourglass_model.add(Layer.Conv2D(65, (3, 3), padding='same', activation='relu'))
    hourglass_model.add(Layer.Conv2D(60, (3, 3),padding='same', activation='relu'))
    hourglass_model.add(Layer.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

    hourglass_model.add(Layer.Conv2D(32, (3, 3), padding='same', activation='relu'))
    hourglass_model.add(Layer.Conv2D(31, (3, 3), padding='same', activation='relu'))
    hourglass_model.add(Layer.Conv2D(30, (3, 3), padding='same', activation='relu'))
    hourglass_model.add(Layer.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

    hourglass_model.add(Layer.Conv2D(25, (3, 3), padding='same', activation='relu'))

    hourglass_model.add(Layer.Flatten())
    hourglass_model.add(Layer.Dense(128))
    hourglass_model.add(Layer.Activation('relu'))
    hourglass_model.add(Layer.Dense(10))
    hourglass_model.add(Layer.Activation('softmax'))
    
    # tuned
    optimizer = hp.Choice('optimizer', values=['adam', 'RMSprop', 'SGD'])
    # tuned
    keras.optimizers.Adam(hp.Choice('learning_rate', [.01, .001, .0001]))

    hourglass_model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return hourglass_model


# Define the Keras TensorBoard callback.
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=100,  # how many model variations to test?
    executions_per_trial=1,  # how many trials per variation? (same model could perform differently)
    directory=LOG_DIR)

tuner.search_space_summary()
    
tuner.search(x=x_train,
            y=train_labels,    
            verbose=2,
            epochs=1,
            batch_size=1024,
            callbacks=[tensorboard_callback],
            validation_data=(x_test, test_labels))

tuner.results_summary()